# Testing BLS Speed Up in Gradient Notebook
Last modified: Nov 18th 2021

In [ ]:
!pip install astropy 

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [ ]:
def run_bls(stitched_lc, 
            bls_params: dict = {'min_per':0.5, 'max_per':15, 
                                'minimum_n_transit':2, 
                                'freq_factor':1,
                                'durations':[0.05, 0.06666666666666668, 
                                             0.08333333333333334, 0.1,
                                             0.11666666666666668, 
                                             0.13333333333333336,
                                             0.15000000000000002, 
                                             0.16666666666666669, 
                                             0.18333333333333335, 0.2], 
                                'objective':'snr'}, 
            compute_stats: bool = True): 

    '''
    args: 
        stitched_lc: list of stitched light curve arrays [time, flux]
        bls_params: params for bls execution. see documentation
        compute_stats: compute statistics on best period/duration combination? default False
    returns: 
        results: the BLS results array 
        bls_model: the BLS model  
        in_transit_mask: mask for the in_transit points. to get not in transit, do ~in_transit_mask
        stats: if compute_stats==True, then the stats on the best period/duration/t0 are returned
    '''

    from astropy.timeseries import BoxLeastSquares
    import numpy as np

    time = stitched_lc['time']
    flux = stitched_lc['flux']

    durations = np.array(bls_params['durations'])
    bls_model = BoxLeastSquares(t=time, y=flux)
    results = bls_model.autopower(durations, frequency_factor=bls_params['freq_factor'], 
                            minimum_period=bls_params['min_per'], 
                            maximum_period=bls_params['max_per'],
                            objective=bls_params['objective'])

    index = np.argmax(results.power)
    period = results.period[index]
    t0 = results.transit_time[index]
    duration = results.duration[index]
    in_transit = bls_model.transit_mask(time, period, 2*duration, t0)
    if compute_stats: 
        stats = bls_model.compute_stats(period, duration, t0)
        return results, bls_model, in_transit, stats

    else: 
        return results, bls_model, in_transit 

In [ ]:
ids = ['TIC_101433378_lc', 'TIC_11071764_lc', 'TIC_110798661_lc', 'TIC_11568422_lc', 'TIC_117380199_lc', 'TIC_11756637_lc', 'TIC_118278380_lc', 'TIC_124481877_lc', 'TIC_12471629_lc', 'TIC_125882822_lc', 'TIC_125977598_lc', 'TIC_129623324_lc', 'TIC_139993838_lc', 'TIC_148841573_lc']

import numpy as np 
import pandas as pd 
import pickle
import time 


times = []

n = 15

for file in ids:  
    with open(file+'.pickle',"rb") as f: 
        content = pickle.load(f)

        stitched_lc = content['stitched_lc'][0]
        start = time.time()

        results, bls_model, in_transit, stats = run_bls(stitched_lc)

        end = time.time()

        times.append(end-start)

print(np.mean(times), n)

4.789911219051906 15


In [ ]:
import os 
counter = 0
for file in os.listdir(): 
    if 'pickle' in file: 
        counter+=1 
print(counter)

92
